# <center> Projet :  Analysez des clients d'un e-commerce </center>



<img src="https://storage.googleapis.com/kaggle-datasets-images/55151/105464/d59245a7014a35a35cc7f7b721de4dae/data-original.png?t=2018-09-21-16-21-21" width=1292>

# <center>Partie : Analyse  exploratoire</center>


### <center>Dataset : OLIST </center>
### <center> https://www.kaggle.com/olistbr/brazilian-ecommerce <center>

Olist souhaite que vous fournissiez à ses équipes d'e-commerce une segmentation des clients qu’elles pourront utiliser au quotidien pour leurs campagnes de communication.

Votre objectif est de comprendre les différents types d’utilisateurs grâce à leur comportement et à leurs données personnelles.

Vous devrez fournir à l’équipe marketing une description actionable de votre segmentation et de sa logique sous-jacente pour une utilisation optimale, ainsi qu’une proposition de contrat de maintenance basée sur une analyse de la stabilité des segments au cours du temps.

# Chargement des données

In [ ]:
#Ci dessous la liste des packages que nous utilsons pour ce projet.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import rpy2
from sklearn.decomposition import PCA
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
import random
random.seed(9001)

#Afficher toute les colonnes
pd.set_option('display.max_columns', None)

print("Les packages sont chargés.")

### Base sur les  id des clients et leurs lieux.

In [ ]:
df_client_lieu=pd.read_csv("olist_customers_dataset.csv",sep=",")
pd.read_csv("olist_customers_dataset.csv",sep=",")

In [ ]:
len(df_client_lieu["customer_id"].unique())

In [ ]:
len(df_client_lieu["customer_unique_id"].unique())

### Base sur les codes géographiques

In [ ]:
df2=pd.read_csv("olist_geolocation_dataset.csv",sep=",")
pd.read_csv("olist_geolocation_dataset.csv",sep=",")

### BASE SUR LES ORDRES QUI RELIENT AUX PRODUITS ET AUX VENDEURS

In [ ]:
df_order_product_seller=pd.read_csv("olist_order_items_dataset.csv",sep=",")
pd.read_csv("olist_order_items_dataset.csv",sep=",")

In [ ]:
fig = px.box(df_order_product_seller, y="price")
fig.show()

In [ ]:
grouped_order = df_order_product_seller.groupby("order_id")
grouped_order

In [ ]:
df_order_product_sellert_sum=grouped_order.max()
df_order_product_seller.iloc[97261]

### BASE SUR LES ORDRES QUI RELIENT AUX PRODUITS ET AUX VENDEURS

In [ ]:
df_paiement=pd.read_csv("olist_order_payments_dataset.csv",sep=",")
pd.read_csv("olist_order_payments_dataset.csv",sep=",")

In [ ]:
len(df_paiement["order_id"].unique())

In [ ]:
df_paiement["order_id"].value_counts()

In [ ]:
grouped_order = df_paiement.groupby("order_id")
grouped_order

In [ ]:
df_paiement_sum=grouped_order.sum()
df_paiement_sum.iloc[9261]

In [ ]:
df_paiement_sum

In [ ]:
df_paiement_mean=grouped_order.mean()
df_paiement_mean.iloc[9261]

In [ ]:
df_paiement_mean

In [ ]:
nombre_articles=df_paiement.order_id.value_counts()
nombre_articles.name="nombre_articles"

In [ ]:
df_nombre_articles = pd.DataFrame(nombre_articles)
df_nombre_articles["order_id"]=df_nombre_articles.index
df_nombre_articles

### BASE SUR LES ORDRES QUI RELIENT AUX PRODUITS ET AUX VENDEURS

In [ ]:
df_review=pd.read_csv("olist_order_reviews_dataset.csv",sep=",")
df_review

### BASE SUR LES ORDRES DONNEES PAR CLIENTS

In [ ]:
df_order_customer=pd.read_csv("olist_orders_dataset.csv",sep=",")
pd.read_csv("olist_orders_dataset.csv",sep=",")

In [ ]:
len(df_order_customer["customer_id"].unique())

### BASE SUR LES PRODUITS

In [ ]:
df_product=pd.read_csv("olist_products_dataset.csv",sep=",")
pd.read_csv("olist_products_dataset.csv",sep=",")

In [ ]:
df_product["product_category_name"].unique()

In [ ]:
df_product["product_category_name"].value_counts()[:10]

### BASE SUR LES VENDEURS

In [ ]:
df_seller=pd.read_csv("olist_sellers_dataset.csv",sep=",")
df_seller

# BASE FINALE

In [ ]:
#base wich link customer and order
df=pd.merge(df_client_lieu,df_order_customer, left_on='customer_id',right_on='customer_id', how='left') 
df

In [ ]:
#df_review
df=pd.merge(df,df_review, left_on='order_id',right_on='order_id', how='inner') 
df

In [ ]:
#df_paiement_mean
df=pd.merge(df,df_paiement_sum, left_on='order_id',right_on='order_id', how='inner') 
df

In [ ]:
#df_order_product_seller
df2=pd.merge(df_order_product_seller,df_product, left_on='product_id',right_on='product_id', how='inner') 
df2

In [ ]:
#df_seller
df2=pd.merge(df2,df_seller, left_on='seller_id',right_on='seller_id', how='inner') 
df2

In [ ]:
grouped_order = df2.groupby("order_id")
grouped_order

In [ ]:
df2_mean=grouped_order.mean()
df2_mean

In [ ]:
#df_paiement_mean
df=pd.merge(df,df2_mean, left_on='order_id',right_on='order_id', how='inner') 
df

In [ ]:
#df_nombre_articles
df=pd.merge(df,df_nombre_articles, left_on='order_id',right_on='order_id', how='inner') 
df=df.sort_values("order_purchase_timestamp")
df

In [ ]:
a=pd.DataFrame(df["customer_unique_id"].value_counts())
a['Client_fidele_numeric'] = np.where(a["customer_unique_id"] > 1, 1, 0)
a['Client_fidele'] = np.where(a["customer_unique_id"] > 1, "client fidèle", "client non fidèle")
b=df.groupby("customer_unique_id")["review_score"].mean()
a=a.rename(columns={"customer_unique_id": "Nombre_commandes"})
a["satisfaction_moyenne"]=b.values
df=pd.merge(df,a, left_on='customer_unique_id',right_on=a.index, how='inner') 
df

In [ ]:
af=df[-df.duplicated( subset=['customer_unique_id'],keep='first')]


In [ ]:
af

# Analyse exploratoire + Feature engineering

Un bon client est un client qui commande beaucoup ou alors qui est satisfait.

In [ ]:
df

In [ ]:
df.payment_sequential.describe()

In [ ]:
df.nombre_articles.describe()

In [ ]:
df["order_purchase_timestamp"]

In [ ]:
df["year"]=df['order_purchase_timestamp'].str[:4]


In [ ]:
plt.figure(figsize=(20,13))
plt.title("Nombre de client par année")
sns.countplot(x='year', data=df)
plt.show()

Regardons la satisfaction client,

In [ ]:
plt.figure(figsize=(20,13))
plt.title("Répartition de la satisfaction client")
sns.countplot(x='review_score', data=df)
plt.show()

In [ ]:
df['Client_satisfaction'] = np.where(df['review_score']>3, "Client satisfait", "Client non satisfait")
df['Client_satisfaction'].value_counts()

In [ ]:
df['Client_satisfaction_numeric'] = np.where(df['review_score']>3, 1, 0)
df['Client_satisfaction_numeric'].value_counts()

Regardons le nombre de commande,

In [ ]:
plt.figure(figsize=(20,13))
plt.title("Répartition du nombre de commande")
sns.countplot(x='nombre_articles', data=df)
plt.show()

In [ ]:
df['Nombre_articles'] = np.where(df['nombre_articles']>1, "Plusieurs articles", "Un article")
df['Nombre_articles'].value_counts()

In [ ]:
df['Nombre_articles_numeric'] = np.where(df['nombre_articles']>1, 1, 0)
df['Nombre_articles_numeric'].value_counts()

Regardons le montant du panier moyen,

In [ ]:
plt.figure(figsize=(20,13))
plt.title("Distribution du prix moyen d'achat")
x = pd.Series(df["price"], name="prix en euro")
ax = sns.distplot(x)
plt.show()

In [ ]:
df["price"].describe()

In [ ]:
df['Client_depensier'] = np.where(df['price']>139, "Client dépensier", "Client non dépensier")
df['Client_depensier'].value_counts()

In [ ]:
df['Client_depensier_numeric'] = np.where(df['price']>139, 1, 0)
df['Client_depensier_numeric'].value_counts()

In [ ]:
plt.figure(figsize=(15,10))
plt.title("Nombre de commande en fonction du prix")
sns.scatterplot(data=df, x="nombre_articles", y="price", hue='Client_satisfaction')
plt.show()

In [ ]:
sns.stripplot(x='review_score', y='price', data=df, jitter=True)
sns.despine()

# Début Modélisation

Pour le feature engenering

### On peut faire une ACP, clustering hiérarchique,k-means,DBSCAN

L'indice de Rand ajusté, ARI nous permettra d'évaluer nos modèles.

In [ ]:
df["volume"]=df["product_length_cm"]*df["product_height_cm"]*df["product_width_cm"]


In [ ]:
df

In [ ]:
df.shape

In [ ]:
df=df.drop(["nombre_articles","satisfaction_moyenne","price","seller_zip_code_prefix","order_item_id","payment_installments","payment_sequential",'review_comment_title','review_comment_message','customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'order_id', 'order_purchase_timestamp', 'order_delivered_carrier_date', 'order_delivered_customer_date','order_estimated_delivery_date','review_id','review_creation_date',"review_answer_timestamp","product_name_lenght","product_description_lenght","product_photos_qty","Nombre_articles_numeric","Client_depensier_numeric"], axis=1)
df=df.drop(["product_length_cm","product_height_cm","product_width_cm"],axis=1)
df
#

In [ ]:
df=df.dropna()
df=df.reset_index()

In [ ]:
df

In [ ]:
df.head(30)

In [ ]:
# Ajout des mensualités, bimensualités et quadrimensualités
for i in range(len(df)):
    df.loc[i,"month"]=df.loc[i,"order_approved_at"][5:7]
    if (df.loc[i,"month"] in {'01','02','03'}) : df.loc[i,"month"] =="Q1"
    if (df.loc[i,"month"] in {"04","05","06"}) : df.loc[i,"month"] =="Q2"
    if (df.loc[i,"month"] in {"07","08","09"}) : df.loc[i,"month"] =="Q3"
    if (df.loc[i,"month"] in {"10","11","12"}) : df.loc[i,"month"] =="Q4"
#####
df.loc[(df["month"] == "01")|(df["month"] == "02"), 'Bimestre'] = "B1"
df.loc[(df["month"] == "03")|(df["month"] == "04"), 'Bimestre'] = "B2"
df.loc[(df["month"] == "06")|(df["month"] == "05"), 'Bimestre'] = "B3"
df.loc[(df["month"] == "08")|(df["month"] == "07"), 'Bimestre'] = "B4"
df.loc[(df["month"] == "10")|(df["month"] == "09"), 'Bimestre'] = "B5"
df.loc[(df["month"] == "12")|(df["month"] == "11"), 'Bimestre'] = "B6"
######
df.loc[(df["Bimestre"] == "B1")|(df["Bimestre"] == "B2"), 'Quadrimestre'] = "Q1"
df.loc[(df["Bimestre"] == "B3")|(df["Bimestre"] == "B4"), 'Quadrimestre'] = "Q2"
df.loc[(df["Bimestre"] == "B5")|(df["Bimestre"] == "B6"), 'Quadrimestre'] = "Q3"

In [ ]:
df.month.unique()
     

In [ ]:
df.loc[1080,:]

In [ ]:

df["month"].unique()

In [ ]:
df.loc[18,:]

In [ ]:
df.order_status.value_counts()

In [ ]:
df=df[df["order_status"]=="delivered"]

In [ ]:
df=df[df["year"]!=2018]
df

In [ ]:

df=df.drop(["index"],axis=1)
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df=df.dropna()
df_pca=df.select_dtypes(include=numerics)
df_pca=df_pca.dropna()
df_pca= df_pca / df_pca.std(axis=0)
df_pca

In [ ]:
n_components = 3
pca = PCA(n_components=n_components)
reduced = pca.fit_transform(df_pca)
 

for i in range(0, n_components):
    df_pca['PC' + str(i + 1)] = reduced[:, i]
 
 
# graphe ébouli
ind = np.arange(0, n_components)
(fig, ax) = plt.subplots(figsize=(8, 6))
sns.pointplot(x=ind, y=pca.explained_variance_ratio_)
ax.set_title('Graphe en ébouli')
ax.set_xticks(ind)
ax.set_xticklabels(ind)
ax.set_xlabel('Nombre composant')
ax.set_ylabel('Variance expliquée')
plt.show()

 
# Cercle corrélation
(fig, ax) = plt.subplots(figsize=(12, 12))
for i in range(0, len(pca.components_)):
    ax.arrow(0, 0,  
             pca.components_[0, i], pca.components_[1, i],  
             head_width=0.1,head_length=0.1)
    plt.text(pca.components_[0, i] + 0.05, pca.components_[1, i] + 0.05, df_pca.columns.values[i])
 
an = np.linspace(0, 2 * np.pi, 100)  
plt.plot(np.cos(an), np.sin(an))
plt.axis('equal')
ax.set_title('Cercle corrélation')
plt.show()

In [ ]:
pca.explained_variance_ratio_.sum()

In [ ]:
pca.explained_variance_ratio_[:2].sum()

In [ ]:
df_pca

In [ ]:
df["PC1"]=df_pca["PC1"]
df["PC2"]=df_pca["PC2"]
df["PC3"]=df_pca["PC3"]

In [ ]:
df

In [ ]:
plt.figure(figsize=(15,10))
plt.title("ACP 2 axes")
sns.scatterplot(data=df, x="PC1", y="PC2",hue="Client_satisfaction")
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
plt.title("ACP 2 axes")
sns.scatterplot(data=df, x="PC1", y="PC2",hue="Client_depensier")
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
plt.title("ACP 2 axes")
sns.scatterplot(data=df, x="PC1", y="PC2",hue="Nombre_commandes")
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
plt.title("ACP 2 axes")
sns.scatterplot(data=df, x="PC1", y="PC2",hue="Client_fidele")
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
plt.title("ACP 2 axes")
sns.scatterplot(data=df, x="PC2", y="PC3",hue="Client_fidele")
plt.show()

In [ ]:
df.Client_fidele.value_counts()

### Pour vérifier notre modélisation on coupera temporellemnt notre base en 2016/2017 et on se reservera 2018 pour notre programme de maintenance

In [ ]:
fig = px.scatter_3d(
    df, x="PC1", y="PC2", z="PC3", color="Client_depensier",
    title="PCA",
)
fig.show()

In [ ]:
fig = px.scatter_3d(
    df, x="PC1", y="PC2", z="PC3", color="Client_satisfaction",
    title="PCA",
)
fig.show()

In [ ]:
fig = px.scatter_3d(
    df, x="PC1", y="PC2", z="PC3", color="Nombre_commandes",
    title="PCA",
)
fig.show()

In [ ]:
fig = px.scatter_3d(
    df, x="PC1", y="PC2", z="PC3", color="Client_fidele",
    title="PCA",
)
fig.show()

# FIN

In [ ]:
df

In [ ]:
df.shape

In [ ]:
#sauveagarde_dataset
df.to_csv("df.csv",index=False)

In [ ]:
PCA

In [ ]:
df_pca.columns

In [ ]:
pca.components_

# Conclusion

On conserve donc 3 axes car on est en mesure d'expliquer 78% de la variance. 